In [ ]:
import gtfparse
import polars as pl
import matplotlib.pyplot as plt
import random
import os
from Bio import SeqIO


def loadingGenome(): # returns gene_groups
    # Load the GTF file
    gtf_file = 'gencode.vM34.chr_patch_hapl_scaff.basic.annotation.gtf'
    gtf_data = gtfparse.read_gtf(gtf_file)

    # Filter to get only exons using Polars
    exons = gtf_data.filter(gtf_data['feature'] == 'exon')

    # Example to display exon positions and gene name using Polars
    exon_details = exons.select(['seqname', 'start', 'end', 'feature', 'gene_name', 'gene_id'])
    exon_details = exon_details.with_columns((exons['end'] - exons['start'] + 1).alias('exon_length'))
    exon_details = exon_details.sort(['gene_name', 'start'])


    # sort exons by gene
    gene_groups = {name: group for name, group in exon_details.group_by("gene_id")}

    print("Successfully Loaded Annotated Genome")
    return gene_groups
  
gene_groups = loadingGenome()


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']
/tmp/ipykernel_70497/1506973251.py:24: DeprecationWarning: `group_by` iteration will change to always return group identifiers as tuples. Pass `by` as a list to silence this warning, e.g. `group_by(['gene_id'])`.
  gene_groups = {name: group for name, group in exon_details.group_by("gene_id")}


Successfully Loaded Annotated Genome


In [ ]:
###################
def createTranscripts(gene_groups):
    # Use exist_ok to avoid error if the directory already exists
    os.makedirs('simulatedmRNAs/', exist_ok=True)

    # load the sequences
    sequences = load_fasta()

    # loop through every gene in gene_groups
    for gene in list(gene_groups.keys()):

        # make a folder to store the mRNA strings for this gene
        loc = os.path.join('simulatedmRNAs', gene)  # Use os.path.join for OS-independent path handling
        os.makedirs(loc, exist_ok=True)  # Use exist_ok here as well

        # get the number of exons for this gene
        exon_count = gene_groups[gene].shape[0]

        # randomly choose how many exons will be present in this transcript
        number_of_exons = exon_count
        # number_of_exons = random.randint(1, exon_count)

        # randomly choose a subset of size number_of_exons
        exon_subset_indices = random.sample(range(exon_count), number_of_exons)
        exon_subset_indices.sort() # make sure the order is sorted

        # initialize exon_boundaries
        exon_boundaries = [] # list of tuples
        
        # store this in our folder
        
        # file_path = os.path.join(loc, 'ans.txt')
        # with open(file_path, 'w') as file:  # Using 'with' ensures the file is closed properly after its suite finishes
        #     for exon_idx in exon_subset_indices: 
        #         start, end = gene_groups[gene]['start'][exon_idx], gene_groups[gene]['end'][exon_idx]

        #         assert(type(start) == type(end) == int)

        #         exon_boundaries.append((start, end))
        #         file.write(str(start) + '/' + str(end) + '\n')
        #     file.write('-')

        # create file to know what sequence this gene comes from (to know what to align it to)
        # file_path = os.path.join(loc, 'sequence_ref.txt')
        # with open(file_path, 'w') as file:
        #     file.write(gene_groups[gene]['seqname'][0])



        # file_path = os.path.join(loc, 'cdna.fa')
        # with open(file_path, 'w') as file:  # Using 'with' ensures the file is closed properly after its suite finishes
        #     file.write('>' + 'cDNA_' + gene + '\n')
        #     sequence = sequences.get(gene_groups[gene]["seqname"][0])

        #     if sequence: 

        #         # ensuring the data we are adding is actually being added
        #         current_length = 0
        #         expected_length = 0

        #         for start, end in exon_boundaries: 
        #             assert(type(start) == type(end) == int)

        #             expected_length += end - start + 1

        #             if (start >= len(sequence)): 
        #                 print("starting index is greater than length of sequence")
        #                 continue
                    
        #             if (end >= len(sequence)):
        #                 print("ending index is greater than the length of sequence")
        #                 continue
                        
        #             # Adjust for Python's 0-based index; subtract 1 from start
        #             exon_seq = sequence[start-1:end]

        #             current_length += len(exon_seq) 

                    
        #             file.write(str(exon_seq))
        #     else: 
        #         print("Sequence type is NONE")
        #         continue
            
        #     if (expected_length != current_length):
        #         print("Expected Length: " + str(expected_length), "Current Length: " + str(current_length))

        #     file.write('\n')

    print("Successfully Created Artificial cDNA strands")
############################################################
createTranscripts(gene_groups)


In [ ]:
gtf_file = 'gencode.vM34.chr_patch_hapl_scaff.basic.annotation.gtf'
gtf_data = gtfparse.read_gtf(gtf_file)